# Поступление в БГУИР 2024
### Анализ приема документов по данным публикуемого мониторинга

## 1. Исходные данные и предпосылки

Расчеты проведены на скорую руку в период приемной кампании для оценки перспектив поступления, без учета какого-либо другого контекста, точных правил применения льгот и других условий зачисления. Метрики рассчитаны изолировано по специальностям без учета общего конкурса. Обработаны только данные о подаче заявлений на дневную форму обучения на бюджет. Исключены "экономические" специальности. 

## 2. Основные наблюдения и выводы

### 2.1. Проходной балл на популярные специальности выше чем в прошлом году 

Предпожительно: 
- задания ЦЭ 2024 проще чем ЦТ 2023, баллы выше у всех желающих поступать;
- в 2024 году медалисты в регионах имеют возможность занять до 80% мест без экзаменов, это ограничивает в региональных вузах общий конкурс и вытесняет абитуриентов с высокими баллами ЦЭ в столицу (*не вполне подтверждается, квота для медалистов не заполнена в некоторых региональных вузах*).

### 2.2. Абитуриенты хотят работать с клавиатурой, а не с паяльником

По уровню конкурса специальности поделились на два кластера: "софт" и "хард". Высок спрос на программистские специальности. Машиностроение, электроника, инженерия -- аутсайдеры по приоритету. Без учета перетока по общему конкурсу на них сложился недобор заявлений. Хуже только педагогика, которая на последнем месте.

Аномально выглядит "Системы и сети инфокоммуникаций". При большом наборе подача документов продвигалась крайне неохотно и только к концу в выравнялась со всеми хард-специальностями. 

## 3. Нереализованные функции

- визуализация,
- объединение с данными подачи на платную форму,
- оценка конкурса и проходного балла с учетом прогноза перераспределения заявлений между специальностями общего конкурса и на платную форму,
- сравнение с данными других вузов.

In [1]:
import pandas as pd
import numpy as np

In [2]:
bsuir_page = pd.read_html('./bsuir_data/dn_budjet.html')    # загружаем страницу с данными
                                                            # исходный url: 'https://abitur.bsuir.by/statistics/2024/schools/group/dn_budjet.html'            
bsuir_table = pd.DataFrame(bsuir_page[0])                   # загрузка таблицы в датафрейм

In [3]:
# bsuir_table.head()

In [4]:
bsuir_table = bsuir_table.drop(bsuir_table.columns[67], axis=1)    # удаляем лишнюю распознанную колонку

In [5]:
bsuir_table = bsuir_table.drop([0])                         # удаляем первую строку
bsuir_table.iat[0, 0] = 'Факультет'                         # переименовываем для названия первой графы
for i in range(3,6):                                        # добавляем сведения обрезанной первой строки в будущий заголовок
    bsuir_table.iat[0, i] = 'Подано ' + bsuir_table.iat[0, i].lower()                                         
for i in range(6,67):                                       # оставляем только верхнюю границу в будущих заголовках когорт баллов
    bsuir_table.iat[0, i] = bsuir_table.iat[0, i].split('-')[1]                           
bsuir_table.columns = bsuir_table.iloc[0]                   # делаем первую строку заголовком

In [6]:
bsuir_table = bsuir_table.dropna(subset=['Специальность'])                      # удаляем промежуточные подзаголовки (строки, у которых пустые графы данных)
bsuir_table = bsuir_table.loc[bsuir_table['Специальность'] != 'Специальность']
bsuir_table = bsuir_table.loc[bsuir_table['Специальность'] != 'ИТОГО']          # удаляем промежуточные заголовки, строку ИТОГО
bsuir_table = bsuir_table.iloc[:-4, :]                                          # удаляем промежуточные итоги и данные экономич. специальнстей в конце таблицы

In [7]:
bsuir_table.iloc[:, 3:67] = bsuir_table.iloc[:, 3:67].fillna('0').astype('int')  # вставляем нули в пустые ячейки, преобразуем данные к численному типу

In [8]:
                                                                                # создаем численный столбец План (места для конкурса)
bsuir_table['Места для конкурса'] = (bsuir_table['План приема'].str
                                                               .strip()
                                                               .apply(lambda x: x.split()[0])
                                                               .astype('int')
                                     - bsuir_table['Подано без вступ.исп.']
                                     - bsuir_table['Подано вне конкурса']
                                    )                                    

In [9]:
                                                                                # Создаем столбец Конкурс
bsuir_table['Конкурс'] = (bsuir_table['Подано всего'] 
                          / bsuir_table['План приема'].str.strip()
                                                      .apply(lambda x: x.split()[0])
                                                      .astype('int')
                          )                                             

In [10]:
ratio = bsuir_table['Конкурс']                                                  # переносим столбец Конкурс вперед
bsuir_table = bsuir_table.drop('Конкурс', axis=1)
bsuir_table.insert(3, 'Конкурс', ratio)                            

In [11]:
bsuir_table = bsuir_table.sort_values(by=['Конкурс'], ascending=[False])        # сортируем специальности в порядке убывания конкурса

In [12]:
                                                                              # считаем средний балл поданных 
                                                                              # (+0.0001 чтобы избежать деления на 0, в числителе тоже ноль в этом случае)
cohorts_columns_slice = slice(7,68)

bsuir_table['Средний балл'] = (np.sum(bsuir_table.iloc[:, cohorts_columns_slice] 
                                * bsuir_table.columns[cohorts_columns_slice].astype('int'), axis=1) 
                                / (np.sum(bsuir_table.iloc[:, cohorts_columns_slice], axis=1) + 0.0001)
                              )
                                             

In [13]:
bsuir_table = bsuir_table.reset_index(drop=True)    

In [14]:
def cutoff_grade(df, plan_series, coh_col_range_start_iloc, coh_col_range_end_iloc):
    '''оценка проходного балла по когортам баллов и плану набора'''
    plan_series.index = df.index                            # приводим метки серии плана приема с к индексу датафрейма
    cumsums = df.iloc[:, coh_col_range_start_iloc:coh_col_range_end_iloc].cumsum(axis=1)    # считаем количество заявлений с накоплением в порядке убывания баллов
    cutoff_series = pd.Series({k:0 for k in df.index})      # создаем серию для проходных баллов с теми же метками что и индекс датафрейма
    for i, row in cumsums.iterrows():                       # ищем проходный балл (последняя по строке когорта баллов, где накопленное количество <= план набора)
        cutoff_series[i] = 0                                 
        for colname, data in row.items():
            if data <= plan_series[i]:                       
                cutoff_series[i] = int(colname)              
            else:
                break
    return cutoff_series

In [15]:
cohorts_start_i = 7
cohorts_end_i = 68

bsuir_table['Проходной балл'] = cutoff_grade(bsuir_table, bsuir_table['Места для конкурса'], cohorts_start_i, cohorts_end_i)  # вставлем проходной балл в таблицу

In [16]:
bsuir_table[['Конкурс', 'Средний балл']] = bsuir_table[['Конкурс', 'Средний балл']].astype(float).round({'Конкурс':2, 'Средний балл':1})

In [17]:
# bsuir_table.to_excel('bsuir_submissions.xlsx')

In [18]:
bsuir_summary = bsuir_table[['Факультет', 'Специальность', 'Подано всего', 'Конкурс', 'Места для конкурса', 'Средний балл', 'Проходной балл']]

In [19]:
bsuir_table.head()

1,Факультет,Специальность,План приема,Конкурс,Подано всего,Подано без вступ.исп.,Подано вне конкурса,400,395,390,...,130,125,120,115,110,105,100,Места для конкурса,Средний балл,Проходной балл
0,Факультет информационных технологий и управления,Информационные системы и технологии,15,6.87,103,4,0,0,5,12,...,0,0,0,0,0,0,0,11,361.4,395
1,Факультет компьютерных систем и сетей,Программная инженерия,46 +9*,5.80,267,27,1,12,35,23,...,0,0,0,0,0,0,0,18,374.1,400
2,Факультет информационных технологий и управления,Искусственный интеллект,56 +4*,2.05,115,1,1,2,2,9,...,0,0,0,0,0,0,0,54,362.6,375
3,Факультет компьютерных систем и сетей,Информатика и технологии программирования,62 +3*,2.05,127,11,2,6,12,15,...,0,0,0,0,0,0,0,49,374.4,385
4,Факультет компьютерного проектирования,Программная инженерия,34 +1*,2.00,68,1,2,0,1,3,...,0,0,0,0,0,0,0,31,349.2,365


In [21]:
bsuir_summary

1,Факультет,Специальность,Подано всего,Конкурс,Места для конкурса,Средний балл,Проходной балл
0,Факультет информационных технологий и управления,Информационные системы и технологии,103,6.87,11,361.4,395
1,Факультет компьютерных систем и сетей,Программная инженерия,267,5.80,18,374.1,400
2,Факультет информационных технологий и управления,Искусственный интеллект,115,2.05,54,362.6,375
3,Факультет компьютерных систем и сетей,Информатика и технологии программирования,127,2.05,49,374.4,385
4,Факультет компьютерного проектирования,Программная инженерия,68,2.00,31,349.2,365
5,Факультет компьютерного проектирования,Информационные системы и технологии,74,1.76,40,350.4,355
6,Факультет информационной безопасности,Информационная безопасность,73,1.74,40,339.2,345
7,Факультет информационных технологий и управления,Электронные системы и технологии,39,1.39,-9,362.5,0
8,Инженерно-экономический факультет,Информационные системы и технологии,91,1.32,67,354.9,340
9,Факультет информационных технологий и управления,Системы управления информацией,76,1.31,58,357.1,340
